<a href="https://colab.research.google.com/github/ravidipu90/self-projects-/blob/main/ravi_assin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
from matplotlib import path
from scipy import ndimage
import os
from scipy import ndimage

In [ ]:
def crop_image(img,pts):
  mask = np.zeros(img.shape, dtype=np.uint8)
  roi_corners = np.array([pts], dtype=np.int32)
  # fill the ROI so it doesn't get wiped out when the mask is applied
  channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
  ignore_mask_color = (255,)*channel_count
  cv2.fillPoly(mask, roi_corners, ignore_mask_color)
  masked_image = cv2.bitwise_and(img, mask)
  return masked_image, mask

def rotate_image(image,mask):
  w,h=image.shape[:-1]
  angle=np.random.randint(low=0,high=45, size=1)[0]
  image = ndimage.rotate(image, angle)
  mask = ndimage.rotate(mask, angle)
  image=cv2.resize(image,(h,w))
  mask=cv2.resize(mask,(h,w))
  return image,mask

def translate_image(img,mask):
  h, w = img.shape[:2]
  t_h = np.random.randint(low=0,high=h//4 ,size=1)[0]
  t_w = np.random.randint(low=0,high=w//4, size=1)[0]
  T = np.float32([[1, 0, t_w], [0, 1, t_h]])
  # We use warpAffine to transform
  # the image using the matrix, T
  img = cv2.warpAffine(img, T, (w, h))
  mask=cv2.warpAffine(mask, T, (w, h))
  return img,mask


def augment_image(item_image, empty_image, annotation, num_output_image,save_dir='results'):
  img=cv2.imread(item_image)
  img_empty1=cv2.imread(empty_image)
  cordinates=json.load(open(annotation))

  crops,masks=[],[]
  for cord in cordinates['20220202-110958.png-1']['regions']:
    x_pts=cord['shape_attributes']['all_points_x']
    y_pts=cord['shape_attributes']['all_points_y']
    pts=[(x,y) for x,y in zip(x_pts,y_pts)]
    crop,mask=crop_image(img,pts)
    crops.append(crop)
    masks.append(mask)

  if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
  images=[]
  for idx in range(num_output_image):
    img_empty=img_empty1.copy()
    for _ in (range(len(crops))):
      crop_idx=np.random.choice(range(len(crops)), size=1)[0]
      crop,mask=crops[crop_idx],masks[crop_idx]
      crop,mask=translate_image(crop,mask)
      crop,mask=rotate_image(crop,mask)
      mask_inv=~mask
      img_empty=cv2.bitwise_and(img_empty, mask_inv)
      img_empty=cv2.bitwise_or(img_empty, crop)

    path=os.path.join(save_dir,'%s.jpg'%(str(idx)))
    cv2.imwrite(path,img_empty)
    images.append(path)
  return images

results=augment_image('item_image.png','empty_image.png','annotation.json',15)
print(results)

FileNotFoundError: ignored

In [ ]:
'''
img=cv2.imread('item_image.png')
img_empty=cv2.imread('empty_image.png')
plt.imshow(img_empty)
plt.show()

def crop_image(img,pts):
  mask = np.zeros(img.shape, dtype=np.uint8)
  roi_corners = np.array([pts], dtype=np.int32)
  # fill the ROI so it doesn't get wiped out when the mask is applied
  channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
  ignore_mask_color = (255,)*channel_count
  cv2.fillPoly(mask, roi_corners, ignore_mask_color)
  masked_image = cv2.bitwise_and(img, mask)
  return masked_image, mask

def paste_crop(img,crop):
  mask = np.zeros(img.shape, dtype=np.uint8)
  roi_corners = np.array([pts], dtype=np.int32)
  # fill the ROI so it doesn't get wiped out when the mask is applied
  channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
  ignore_mask_color = (255,)*channel_count
  cv2.fillPoly(mask, roi_corners, ignore_mask_color)
  masked_image = cv2.bitwise_and(img, mask)
  return masked_image


crops,masks=[],[]
cordinates=json.load(open('annotation.json'))
#print(cordinates)

for cord in cordinates['20220202-110958.png-1']['regions']:
  x_pts=cord['shape_attributes']['all_points_x']
  y_pts=cord['shape_attributes']['all_points_y']
  pts=[(x,y) for x,y in zip(x_pts,y_pts)]
  crop,mask=crop_image(img,pts)
  crops.append(crop)
  masks.append(mask)
  plt.imshow(crop)
  plt.show()
  plt.imshow(mask)
  plt.show()

'''

In [ ]:
'''

for _ in range(10):
  crop_idx=np.random.choice(range(len(crops)), size=1)[0]
  crop,mask=crops[crop_idx],masks[crop_idx]
  crop,mask=translate_image(crop,mask)
  crop,mask=rotate_image(crop,mask)
  mask_inv=~mask
  img_empty=cv2.bitwise_and(img_empty, mask_inv)
  img_empty=cv2.bitwise_or(img_empty, crop)

  plt.imshow(crop)
  plt.show()
  plt.imshow(img_empty)
  plt.show()
'''